In [1]:
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

if os.getcwd().endswith('notebook'):
    os.chdir('..')
print(f'current working directory: {os.getcwd()}')

current working directory: /Users/howechen/Project/ntu_sd6103_team_project/ntu_sd6103_data_systems_team_project


In [84]:
# load data
pub_csv_path = "../ntu_sd6103_team_project_data/csv_files/publications_raw.csv"
author_csv_path = "../ntu_sd6103_team_project_data/csv_files/authors.csv"
author_pub_csv_path = "../ntu_sd6103_team_project_data/csv_files/author_publications.csv"
jrnls_csv_path = "../ntu_sd6103_team_project_data/csv_files/journals.csv"
conf_csv_path = "../ntu_sd6103_team_project_data/csv_files/conferences.csv"
editors_csv_path = "../ntu_sd6103_team_project_data/csv_files/editors.csv"
editor_pub_csv_path = "../ntu_sd6103_team_project_data/csv_files/editor_publications.csv"
publisher_csv_path = "../ntu_sd6103_team_project_data/csv_files/publishers.csv"
posed_conf_csv_path = '../ntu_sd6103_team_project_data/csv_files/processed/conferences.csv'
posed_jrnls_csv_path = '../ntu_sd6103_team_project_data/csv_files/processed/journals.csv'
posed_publisher_csv_path = '../ntu_sd6103_team_project_data/csv_files/processed/publishers.csv'
pub_df = pd.read_csv(pub_csv_path)
au_pub_df = pd.read_csv(author_pub_csv_path)
au_df = pd.read_csv(author_csv_path)
jrnls_df = pd.read_csv(jrnls_csv_path)
conf_df = pd.read_csv(conf_csv_path)
editors_df = pd.read_csv(editors_csv_path)
ed_pub_df = pd.read_csv(editor_pub_csv_path)
publisher_df = pd.read_csv(publisher_csv_path)
posed_conf_df = pd.read_csv(posed_conf_csv_path)
posed_jrnls_df = pd.read_csv(posed_jrnls_csv_path)
posed_publisher_df = pd.read_csv(posed_publisher_csv_path)

In [24]:
df = pd.read_csv('../ntu_sd6103_team_project_data/csv_files/publications.csv')
df['journal_id'].dtype, df['publisher_id'].dtype

(dtype('int64'), dtype('int64'))

In [97]:
pub_df[pub_df['pub_id']=='conf/aaai/ZhangZZL24']

,type,pub_id,mdate,title,year,key_source,key_name,volume,publisher,journal,number,url,ee,key_name_striped
312874,inproceedings,conf/aaai/ZhangZZL24,2024-08-01,HONGAT: Graph Attention Networks in the Presen...,24.0,conf,aaai/ZhangZZL24,NaN,NaN,NaN,NaN,db/conf/aaai/aaai2024.html#ZhangZZL24,https://doi.org/10.1609/aaai.v38i15.29615,aaai24


In [98]:
temp = pub_df[pub_df['year']<1000]
temp['year'] = temp['year'].apply(lambda x: x+2000 if x <= 25 else x)
temp['year'] = temp['year'].apply(lambda x: x+1900 if (x>25) and (x <=100) else x)
temp['year'] = temp['year'].apply(lambda x: x+1000 if (x >500) and (x <1000) else x)
temp['year'].max(), temp['year'].min()

(np.float64(2024.0), np.float64(1980.0))

In [100]:
pub_df.loc[temp.index, 'year'] = temp.loc[temp.index, 'year']

In [101]:
pub_df['year'].min(), pub_df['year'].max()

(np.float64(1936.0), np.float64(2025.0))

In [102]:
pub_df['key_name_striped'] = pub_df['key_name'].str.split('/').str[0]
pub_df['year'] = pub_df['year'].astype('Int64')
pub_df['key_name_striped'] = pub_df.apply(lambda x: str(x['key_name_striped'])+str(x['year']), axis=1)
pub_df.head()

,type,pub_id,mdate,title,year,key_source,key_name,volume,publisher,journal,number,url,ee,key_name_striped
0,incollection,reference/vision/X14bd,2017-07-12,Curvedness.,2014,reference,vision/X14bd,NaN,NaN,NaN,NaN,db/reference/vision/vision2014.html#X14bd,https://doi.org/10.1007/978-0-387-31439-6_100117,vision2014
1,incollection,reference/vision/Singh14,2019-01-09,Transparency and Translucency.,2014,reference,vision/Singh14,NaN,NaN,NaN,NaN,db/reference/vision/vision2014.html#Singh14,https://doi.org/10.1007/978-0-387-31439-6_559,vision2014
2,incollection,reference/vision/X14ii,2017-07-12,Surface Orientation Histogram (Discrete Versio...,2014,reference,vision/X14ii,NaN,NaN,NaN,NaN,db/reference/vision/vision2014.html#X14ii,https://doi.org/10.1007/978-0-387-31439-6_100229,vision2014
3,incollection,reference/vision/X14m,2017-07-12,Appearance Scanning.,2014,reference,vision/X14m,NaN,NaN,NaN,NaN,db/reference/vision/vision2014.html#X14m,https://doi.org/10.1007/978-0-387-31439-6_100189,vision2014
4,incollection,reference/vision/Wong14,2017-07-12,Image-Based Lighting.,2014,reference,vision/Wong14,NaN,NaN,NaN,NaN,db/reference/vision/vision2014.html#Wong14,https://doi.org/10.1007/978-0-387-31439-6_14,vision2014


In [105]:
pub_df.to_csv('../ntu_sd6103_team_project_data/csv_files/publications_raw.csv', index=False)

In [106]:
new_pub_df = pub_df.loc[:, ['type','pub_id','mdate','title','year','volume','publisher','journal','number','url','ee']]
new_pub_df = pd.merge(new_pub_df, posed_publisher_df, left_on='publisher', right_on='name', how='left')
new_pub_df = new_pub_df.drop(columns=['name'])
new_pub_df = pd.merge(new_pub_df, posed_jrnls_df, left_on='journal', right_on='name', how='left')
new_pub_df = new_pub_df.drop(columns=['name'])
new_pub_df = new_pub_df.loc[:, ['type','pub_id','mdate','title','year','volume','publisher_id','publisher','journal','journal_id','number','url','ee']]
new_pub_df['journal_id'].fillna(0, inplace=True)
new_pub_df['publisher_id'].fillna(0, inplace=True)
new_pub_df['year'].fillna(0, inplace=True)
new_pub_df['journal_id'] = new_pub_df['journal_id'].astype('Int64')
new_pub_df['publisher_id'] = new_pub_df['publisher_id'].astype('Int64')
new_pub_df['year'] = new_pub_df['year'].astype('Int64')
# new_pub_df['volume'] = new_pub_df['volume'].astype('Int64')
# new_pub_df['number'] = new_pub_df['number'].astype('Int64')
new_pub_df.head()

,type,pub_id,mdate,title,year,volume,publisher_id,publisher,journal,journal_id,number,url,ee
0,incollection,reference/vision/X14bd,2017-07-12,Curvedness.,2014,NaN,0,NaN,NaN,0,NaN,db/reference/vision/vision2014.html#X14bd,https://doi.org/10.1007/978-0-387-31439-6_100117
1,incollection,reference/vision/Singh14,2019-01-09,Transparency and Translucency.,2014,NaN,0,NaN,NaN,0,NaN,db/reference/vision/vision2014.html#Singh14,https://doi.org/10.1007/978-0-387-31439-6_559
2,incollection,reference/vision/X14ii,2017-07-12,Surface Orientation Histogram (Discrete Versio...,2014,NaN,0,NaN,NaN,0,NaN,db/reference/vision/vision2014.html#X14ii,https://doi.org/10.1007/978-0-387-31439-6_100229
3,incollection,reference/vision/X14m,2017-07-12,Appearance Scanning.,2014,NaN,0,NaN,NaN,0,NaN,db/reference/vision/vision2014.html#X14m,https://doi.org/10.1007/978-0-387-31439-6_100189
4,incollection,reference/vision/Wong14,2017-07-12,Image-Based Lighting.,2014,NaN,0,NaN,NaN,0,NaN,db/reference/vision/vision2014.html#Wong14,https://doi.org/10.1007/978-0-387-31439-6_14


In [107]:
new_pub_df.loc[:,['pub_id','type','title','year','mdate','publisher','publisher_id','journal','journal_id','volume','number','url','ee']].to_csv('../ntu_sd6103_team_project_data/csv_files/publications.csv', index=False)

In [127]:
len = new_pub_df['ee'].str.len()
len.max()

np.float64(263.0)

In [77]:
pub_df[(pub_df['key_source']=='conf') & (pub_df['type'] != 'proceedings')]

,type,pub_id,mdate,year,key_source,key_name,volume,publisher,journal,number,url,ee,key_name_striped
58246,inproceedings,conf/dac/NewtonRMSB00,2018-11-06,2000,conf,dac/NewtonRMSB00,NaN,NaN,NaN,NaN,db/conf/dac/dac2000.html#NewtonRMSB00,https://doi.org/10.1145/337292.337434,dac2000
58247,inproceedings,conf/dac/Jabri88,2017-03-16,1988,conf,dac/Jabri88,NaN,NaN,NaN,NaN,db/conf/dac/dac88.html#Jabri88,http://portal.acm.org/citation.cfm?id=285730.2...,dac1988
58248,inproceedings,conf/dac/SixCRM86,2018-11-06,1986,conf,dac/SixCRM86,NaN,NaN,NaN,NaN,db/conf/dac/dac1986.html#SixCRM86,https://doi.org/10.1145/318013.318151,dac1986
58249,inproceedings,conf/dac/YalcinMPBS99,2018-11-06,1999,conf,dac/YalcinMPBS99,NaN,NaN,NaN,NaN,db/conf/dac/dac99.html#YalcinMPBS99,https://doi.org/10.1145/309847.310045,dac1999
58250,inproceedings,conf/dac/LiuPS98,2018-11-06,1998,conf,dac/LiuPS98,NaN,NaN,NaN,NaN,db/conf/dac/dac98.html#LiuPS98,https://doi.org/10.1145/277044.277174,dac1998
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3678629,inproceedings,conf/sgiot/0004WZZ23,2024-04-19,2023,conf,sgiot/0004WZZ23,NaN,NaN,NaN,NaN,db/conf/sgiot/sgiot2023.html#0004WZZ23,https://doi.org/10.1007/978-3-031-55976-1_4,sgiot2023
3678630,inproceedings,conf/sgiot/ChangJYCC23,2024-04-19,2023,conf,sgiot/ChangJYCC23,NaN,NaN,NaN,NaN,db/conf/sgiot/sgiot2023.html#ChangJYCC23,https://doi.org/10.1007/978-3-031-55976-1_14,sgiot2023
3678631,inproceedings,conf/sgiot/KhandakerHA18,2023-02-12,2018,conf,sgiot/KhandakerHA18,NaN,NaN,NaN,NaN,db/conf/sgiot/sgiot2018.html#KhandakerHA18,https://doi.org/10.1007/978-3-030-05928-6_11,sgiot2018
3678632,inproceedings,conf/sgiot/GaoWYLY22,2023-09-05,2022,conf,sgiot/GaoWYLY22,NaN,NaN,NaN,NaN,db/conf/sgiot/sgiot2022.html#GaoWYLY22,https://doi.org/10.1007/978-3-031-31275-5_21,sgiot2022


In [110]:
pub_df[pub_df['key_source']=='conf'].shape[0], pub_df[pub_df['key_source']=='conf']['key_name_striped'].nunique(), posed_conf_df.shape[0]

(3620388, 48912, 46720)

In [128]:
new_conf_df = pub_df[(pub_df['key_source']=='conf') & (pub_df['type'] == 'proceedings')].loc[:, ['key_name_striped', 'year', 'publisher']].copy()
new_conf_df.drop_duplicates(inplace=True)

In [129]:
new_conf_publisher_df = pub_df[(pub_df['key_source']=='conf') & (pub_df['type'] == 'proceedings')].loc[:, ['key_name_striped', 'year', 'publisher']].copy()
new_conf_publisher_df.drop_duplicates(inplace=True)
new_conf_publisher_df.dropna(subset=['publisher'], inplace=True)
new_conf_publisher_df = pd.merge(new_conf_publisher_df, posed_publisher_df, left_on='publisher', right_on='name', how='left')
new_conf_publisher_df.drop(columns=['name'], inplace=True)
new_conf_publisher_df.rename(columns={'key_name_striped': 'conf_name'},inplace=True)
new_conf_publisher_df['publisher_id'] = new_conf_publisher_df['publisher_id'].astype('Int64')

In [130]:
new_conf_publisher_df.head()

,conf_name,year,publisher,publisher_id
0,dac1995,1995,ACM Press,44
1,dac2010,2010,ACM,45
2,dac2004,2004,ACM,45
3,dac1964,1964,ACM,45
4,dac1986,1986,IEEE Computer Society Press,46


In [131]:
new_conf_df = new_conf_publisher_df.loc[:,['conf_name', 'year']]
new_conf_df.drop_duplicates(inplace=True)
new_conf_df.reset_index(drop=True, inplace=True)
new_conf_df['conf_id'] = new_conf_df.index + 1
new_conf_df.shape[0]

44738

In [132]:
new_conf_df.head()

,conf_name,year,conf_id
0,dac1995,1995,1
1,dac2010,2010,2
2,dac2004,2004,3
3,dac1964,1964,4
4,dac1986,1986,5


In [133]:
len = new_conf_df['conf_name'].str.len()
len.max()

np.int64(25)

In [134]:
new_conf_df.loc[:,['conf_id','conf_name']].to_csv('../ntu_sd6103_team_project_data/csv_files/conferences.csv', index=False)

In [135]:
new_conf_publisher_df.head()

,conf_name,year,publisher,publisher_id
0,dac1995,1995,ACM Press,44
1,dac2010,2010,ACM,45
2,dac2004,2004,ACM,45
3,dac1964,1964,ACM,45
4,dac1986,1986,IEEE Computer Society Press,46


In [136]:
new_conf_publisher_df = pd.merge(new_conf_publisher_df, new_conf_df, on=['conf_name','year'], how='left')
# new_conf_publisher_df.drop(columns=['conf_name'], inplace=True)
new_conf_publisher_df.loc[:,['conf_id','publisher_id','year']].to_csv('../ntu_sd6103_team_project_data/csv_files/conference_publisher.csv', index=False)

In [137]:
temp = pd.merge(pub_df, new_conf_df, left_on='key_name_striped', right_on='conf_name', how='left')
temp.drop(columns=['key_name_striped','year_y'], inplace=True)
temp.rename(columns={'year_x':'year'}, inplace=True)
temp['conf_id'].fillna(0, inplace=True)
temp['conf_id'] = temp['conf_id'].astype('Int64')
temp.head()

,type,pub_id,mdate,title,year,key_source,key_name,volume,publisher,journal,number,url,ee,conf_name,conf_id
0,incollection,reference/vision/X14bd,2017-07-12,Curvedness.,2014,reference,vision/X14bd,NaN,NaN,NaN,NaN,db/reference/vision/vision2014.html#X14bd,https://doi.org/10.1007/978-0-387-31439-6_100117,NaN,0
1,incollection,reference/vision/Singh14,2019-01-09,Transparency and Translucency.,2014,reference,vision/Singh14,NaN,NaN,NaN,NaN,db/reference/vision/vision2014.html#Singh14,https://doi.org/10.1007/978-0-387-31439-6_559,NaN,0
2,incollection,reference/vision/X14ii,2017-07-12,Surface Orientation Histogram (Discrete Versio...,2014,reference,vision/X14ii,NaN,NaN,NaN,NaN,db/reference/vision/vision2014.html#X14ii,https://doi.org/10.1007/978-0-387-31439-6_100229,NaN,0
3,incollection,reference/vision/X14m,2017-07-12,Appearance Scanning.,2014,reference,vision/X14m,NaN,NaN,NaN,NaN,db/reference/vision/vision2014.html#X14m,https://doi.org/10.1007/978-0-387-31439-6_100189,NaN,0
4,incollection,reference/vision/Wong14,2017-07-12,Image-Based Lighting.,2014,reference,vision/Wong14,NaN,NaN,NaN,NaN,db/reference/vision/vision2014.html#Wong14,https://doi.org/10.1007/978-0-387-31439-6_14,NaN,0


In [138]:
pub_conf_df = temp[temp['key_source']=='conf'].copy()
pub_conf_df = pub_conf_df.loc[:,['pub_id','year','conf_id','conf_name']]
pub_conf_df['conf_id'] = pub_conf_df['conf_id'].astype('Int64')
pub_conf_df.loc[:,['pub_id','conf_id']].to_csv('../ntu_sd6103_team_project_data/csv_files/publication_conferences.csv', index=False)

In [139]:
pub_conf_df.loc[:,['pub_id','conf_id','conf_name','year']].head()

,pub_id,conf_id,conf_name,year
58246,conf/dac/NewtonRMSB00,21,dac2000,2000
58247,conf/dac/Jabri88,51,dac1988,1988
58248,conf/dac/SixCRM86,5,dac1986,1986
58249,conf/dac/YalcinMPBS99,23,dac1999,1999
58250,conf/dac/LiuPS98,43,dac1998,1998


In [140]:
temp = pd.merge(pub_df, posed_jrnls_df, left_on='journal', right_on='name', how='left')
temp.drop(columns=['name'], inplace=True)
temp.rename(columns={'journal':'journal_name'}, inplace=True)
temp.head()

,type,pub_id,mdate,title,year,key_source,key_name,volume,publisher,journal_name,number,url,ee,key_name_striped,journal_id
0,incollection,reference/vision/X14bd,2017-07-12,Curvedness.,2014,reference,vision/X14bd,NaN,NaN,NaN,NaN,db/reference/vision/vision2014.html#X14bd,https://doi.org/10.1007/978-0-387-31439-6_100117,vision2014,NaN
1,incollection,reference/vision/Singh14,2019-01-09,Transparency and Translucency.,2014,reference,vision/Singh14,NaN,NaN,NaN,NaN,db/reference/vision/vision2014.html#Singh14,https://doi.org/10.1007/978-0-387-31439-6_559,vision2014,NaN
2,incollection,reference/vision/X14ii,2017-07-12,Surface Orientation Histogram (Discrete Versio...,2014,reference,vision/X14ii,NaN,NaN,NaN,NaN,db/reference/vision/vision2014.html#X14ii,https://doi.org/10.1007/978-0-387-31439-6_100229,vision2014,NaN
3,incollection,reference/vision/X14m,2017-07-12,Appearance Scanning.,2014,reference,vision/X14m,NaN,NaN,NaN,NaN,db/reference/vision/vision2014.html#X14m,https://doi.org/10.1007/978-0-387-31439-6_100189,vision2014,NaN
4,incollection,reference/vision/Wong14,2017-07-12,Image-Based Lighting.,2014,reference,vision/Wong14,NaN,NaN,NaN,NaN,db/reference/vision/vision2014.html#Wong14,https://doi.org/10.1007/978-0-387-31439-6_14,vision2014,NaN


In [141]:
pub_journal_df = temp.loc[:, ['pub_id', 'journal_id']]
pub_journal_df.dropna(subset=['journal_id'], inplace=True)
pub_journal_df['journal_id'] = pub_journal_df['journal_id'].astype('Int64')
pub_journal_df.to_csv('../ntu_sd6103_team_project_data/csv_files/publication_journals.csv', index=False)

In [124]:
au_pub_df.shape, au_df.shape

((28548535, 3), (4844862, 2))

In [125]:
editors_df.shape, ed_pub_df.shape

((56112, 2), (152711, 3))

In [42]:
au_pub_df[au_pub_df.duplicated(subset=['author_name', 'pub_id'])]

,author_id,author_name,pub_id
9974,NaN,n-Altamirano,reference/npe/Villen-AltamiranoV11
12499,NaN,nez,reference/assisted/ObachBLMAIMRP12
18380,NaN,a,reference/ai/Romero-GarciaFSRBHS09
18474,NaN,as Flores,reference/ai/FloresFCM09a
18499,NaN,as Flores,reference/ai/FloresFCM09
...,...,...,...
28409210,NaN,inen,books/daglib/p/KamarainenHP11
28413446,NaN,rstad,books/siam/95/BjorstadK95
28414121,NaN,rklund,books/ems/21/BjorklundBM21
28417103,NaN,ndez,books/idea/becker2003/FernandezFG03


In [28]:
au_df[(au_df['name']=='a') & (-au_df['author_id'].isna())]

,author_id,name
12959,0000-0002-4531-1473,a
21325,0000-0002-7350-6067,a
22452,0000-0002-3422-7225,a
27639,0000-0002-1513-1527,a
28896,0000-0002-4949-2179,a
...,...,...
28406028,0000-0002-4882-9497,a
28408738,0000-0001-9216-9695,a
28408788,0000-0003-0452-9721,a
28410154,0000-0003-4670-7629,a


In [29]:
au_df[(au_df['name']=='a') & (au_df['author_id'].isna())]

,author_id,name
668,NaN,a
709,NaN,a
741,NaN,a
924,NaN,a
7188,NaN,a
...,...,...
28549011,NaN,a
28549024,NaN,a
28549103,NaN,a
28552830,NaN,a


In [143]:
au_pub_df.loc[:,['author_id','pub_id']].to_csv(author_pub_csv_path,index=False)

In [43]:
for df in [pub_df, au_pub_df, au_df, jrnls_df, conf_df, editors_df, ed_pub_df, publisher_df]:
    # print(df.info())
    # print(df.head())
    print(df.shape)
    print('------------------------------------')

(11179600, 12)
------------------------------------
(28556780, 3)
------------------------------------
(28556780, 2)
------------------------------------
(3669449, 2)
------------------------------------
(0, 2)
------------------------------------
(152730, 2)
------------------------------------
(152730, 3)
------------------------------------
(93944, 2)
------------------------------------


In [3]:
# conference list
pub_df[(pub_df['key_source'] == 'conf') & (pub_df['type'] == 'proceedings')]

,type,pub_id,mdate,year,key_source,key_name,volume,publisher,journal,number,url,ee
58289,proceedings,conf/dac/1995,2011-11-30,1995.0,conf,dac/1995,NaN,ACM Press,NaN,NaN,db/conf/dac/dac95.html,http://dl.acm.org/citation.cfm?id=217474
58331,proceedings,conf/dac/2010,2011-11-30,2010.0,conf,dac/2010,NaN,ACM,NaN,NaN,db/conf/dac/dac2010.html,http://dl.acm.org/citation.cfm?id=1837274
58449,proceedings,conf/dac/2004,2011-11-30,2004.0,conf,dac/2004,NaN,ACM,NaN,NaN,db/conf/dac/dac2004.html,http://dl.acm.org/citation.cfm?id=996566
58583,proceedings,conf/dac/1964,2012-03-04,1964.0,conf,dac/1964,NaN,ACM,NaN,NaN,db/conf/dac/dac1964.html,http://dl.acm.org/citation.cfm?id=800265
58801,proceedings,conf/dac/1986,2018-06-19,1986.0,conf,dac/1986,NaN,IEEE Computer Society Press,NaN,NaN,db/conf/dac/dac1986.html,http://dl.acm.org/citation.cfm?id=318013
...,...,...,...,...,...,...,...,...,...,...,...,...
3678517,proceedings,conf/sgiot/2021,2023-01-12,2022.0,conf,sgiot/2021,447,Springer,NaN,NaN,db/conf/sgiot/sgiot2021.html,https://doi.org/10.1007/978-3-031-20398-5
3678554,proceedings,conf/sgiot/2023,2024-04-19,2024.0,conf,sgiot/2023,557,Springer,NaN,NaN,db/conf/sgiot/sgiot2023.html,https://doi.org/10.1007/978-3-031-55976-1
3678575,proceedings,conf/sgiot/2020,2023-01-12,2021.0,conf,sgiot/2020,354,Springer,NaN,NaN,db/conf/sgiot/sgiot2020.html,https://doi.org/10.1007/978-3-030-69514-9
3678581,proceedings,conf/sgiot/2018,2023-01-12,2019.0,conf,sgiot/2018,256,Springer,NaN,NaN,db/conf/sgiot/sgiot2018.html,https://doi.org/10.1007/978-3-030-05928-6


In [4]:
pub_df[pub_df['type'] == 'proceedings']

,type,pub_id,mdate,year,key_source,key_name,volume,publisher,journal,number,url,ee
5760,proceedings,reference/med/2013,2017-05-16,2013.0,reference,med/2013,NaN,Springer,NaN,NaN,db/reference/med/med2013.html,https://doi.org/10.1007/978-1-4614-8495-0
15227,proceedings,reference/genetic/2015,2020-03-27,2015.0,reference,genetic/2015,NaN,Springer,NaN,NaN,db/reference/genetic/genetic2015.html,https://doi.org/10.1007/978-3-319-20883-1
39686,proceedings,series/isrl/97,2017-05-16,2016.0,series,isrl/97,97,Springer,NaN,NaN,db/series/isrl/isrl97.html,https://doi.org/10.1007/978-3-319-24499-0
42117,proceedings,series/sapere/2013-5,2019-09-06,2013.0,series,sapere/2013-5,5,Springer,NaN,NaN,db/series/sapere/sapere5.html,https://doi.org/10.1007/978-3-642-31674-6
42196,proceedings,series/ssw/49,2021-04-19,2020.0,series,ssw/49,49,IOS Press,NaN,NaN,db/series/ssw/ssw49.html,https://doi.org/10.3233/SSW49
...,...,...,...,...,...,...,...,...,...,...,...,...
11013657,proceedings,books/sp/BGPP2015,2017-05-16,2015.0,books,sp/BGPP2015,NaN,Springer,NaN,NaN,db/books/collections/BGPP2015.html,https://doi.org/10.1007/978-3-319-14998-1
11013706,proceedings,books/sp/2015LKJK,2021-04-29,2015.0,books,sp/2015LKJK,NaN,Springer,NaN,NaN,db/books/collections/LKJK2015.html,https://doi.org/10.1007/978-3-319-19291-8
11019281,proceedings,books/sp/BBH2015,2021-07-25,2015.0,books,sp/BBH2015,NaN,Springer,NaN,NaN,db/books/collections/BBH2015.html,https://doi.org/10.1007/978-3-319-17963-6
11032083,proceedings,books/daglib/0028528,2021-04-29,2011.0,books,daglib/0028528,NaN,Springer,NaN,NaN,db/books/collections/MLNM2011.html,https://doi.org/10.1007/978-1-4419-7934-6


In [5]:
conf_df = pub_df[(pub_df['key_source'] == 'conf') & (pub_df['type'] == 'proceedings')].loc[:, ['key_name']]

In [6]:
conf_df.head()

,key_name
58289,dac/1995
58331,dac/2010
58449,dac/2004
58583,dac/1964
58801,dac/1986


In [7]:
conf_df['key_name_split'] = conf_df['key_name'].apply(lambda x: x.split('/'))

In [9]:
conf_df['name'] = conf_df['key_name_split'].apply(lambda x: str(x[0].upper() + x[1]))

In [ ]:
conf_df.drop(columns=['key_name_split', 'key_name'], inplace=True)
conf_df.reset_index(drop=False, inplace=True)
conf_df.rename(columns={'index': 'conf_id'}, inplace=True)

In [20]:
conf_df.head()

,conf_id,name
0,58289,DAC1995
1,58331,DAC2010
2,58449,DAC2004
3,58583,DAC1964
4,58801,DAC1986
